# Advanced ML, Recomendation project : 

### Description of the General topic:  
 
A recommendation system is a class of machine learning tools designed to suggest 
relevant items to users based on their preferences, behaviors, and other users’ 
activities. They are widely used across e-commerce, streaming platforms, social 
media, and online advertising, aiming to enhance user experience by delivering 
personalized content or product suggestions.

### Flow of the Code Project (All in Python)  :  
 
**Data Preprocessing:** 
- Preprocess textual data using tokenization, stemming, and vectorization 
techniques like TF-IDF or word embeddings. 
 
**Building Collaborative Filtering Models:** 
- Implement SVD for matrix factorization. 
- Build user-based and item-based collaborative filtering models using 
libraries like Scikit-Learn. 
 
**Building Content-Based Filtering Models:**
- Create item profiles using metadata. 
- Use cosine similarity or neural embeddings to identify similar items. 
 
**Combine with Hybrid Techniques:** 
- Experiment with hybrid models (weighted hybrid, feature-augmented 
collaborative filtering, etc.) to combine collaborative and content-based 
methods. 
- Train deep hybrid models if using neural networks, concatenating 
collaborative and content-based embeddings as input. 

**Visualization:**  
- Récapitulatifs des des résultats et visualisation par des graphiques dans 
la mesure du possible 

In [1]:
# Import of users data : 
import pandas as pd
import json

file = "/Users/aminerazig/Desktop/ENSAE 3A/ADVANCED ML/Advanced ML-project/DATA/Health_and_Personal_Care.jsonl"

with open(file, 'r') as file:
    data = [json.loads(line) for line in file]

# first 1000 products
products_1000_usersdata = data[:1000]

In [6]:
len(products_1000_usersdata)

1000

In [18]:
# import of products metadata : 

products_1000_metadata = []
file_metadata = "/Users/aminerazig/Desktop/ENSAE 3A/ADVANCED ML/Advanced ML-project/DATA/meta_Health_and_Personal_Care.jsonl"

with open(file_metadata, 'r') as file:
    for i, line in enumerate(file):
        if i >= 1000:  
            break
        products_1000_metadata.append(json.loads(line))

In [22]:
### AFFICHAGE DE QUELQUES IMAGES  : 
import json
import random
import requests
from PIL import Image
from io import BytesIO



def get_random_products_with_images(products, num_products=90):
    products_with_images = [p for p in products if p.get('images') and len(p['images']) > 0]
    return random.sample(products_with_images, min(num_products, len(products_with_images)))


def fetch_and_resize_image(url, size=(30, 30)):
    try:
        response = requests.get(url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        return img.resize(size)
    except Exception as e:
        print(f"Erreur lors du téléchargement de l'image : {e}")
        return None

# mosaïque
def create_mosaic(images, grid_size=(10, 9), image_size=(30, 30)):
    mosaic = Image.new('RGB', (grid_size[0] * image_size[0], grid_size[1] * image_size[1]))
    for idx, img in enumerate(images):
        if img:
            x = (idx % grid_size[0]) * image_size[0]
            y = (idx // grid_size[0]) * image_size[1]
            mosaic.paste(img, (x, y))
    mosaic.show()
    return mosaic


selected_products = get_random_products_with_images(products_1000_metadata)
image_urls = [p['images'][0]['large'] for p in selected_products]

images = [fetch_and_resize_image(url) for url in image_urls]
mosaic = create_mosaic(images)

# Data Fields

## For User Reviews

| Field              | Type   | Explanation                                                                                                                                                     |
|--------------------|--------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `rating`           | float  | Rating of the product (from 1.0 to 5.0).                                                                                                                        |
| `title`            | str    | Title of the user review.                                                                                                                                       |
| `text`             | str    | Text body of the user review.                                                                                                                                   |
| `images`           | list   | Images that users post after they have received the product. Each image has different sizes (small, medium, large), represented by `small_image_url`, `medium_image_url`, and `large_image_url`. |
| `asin`             | str    | ID of the product.                                                                                                                                              |
| `parent_asin`      | str    | Parent ID of the product. Note: Products with different colors, styles, sizes usually belong to the same parent ID. The “asin” in previous Amazon datasets is actually the parent ID. Please use parent ID to find product meta. |
| `user_id`          | str    | ID of the reviewer.                                                                                                                                             |
| `timestamp`        | int    | Time of the review (unix time).                                                                                                                                 |
| `verified_purchase`| bool   | User purchase verification.                                                                                                                                     |
| `helpful_vote`     | int    | Helpful votes of the review.                                                                                                                                    |


In [17]:
import EDA_functions
image_url = "https://images-na.ssl-images-amazon.com/images/I/71DFEoJ+Z9L._SL256_.jpg"
EDA_functions.show_image(image_url)

# I- Collaborative Filtering

Collaborative filtering recommends products to users based on the behavior of other users with similar preferences. CF methods work on the principle that users who agreed on items in the past are likely to agree again. There are two main types:

### a) User-Based Collaborative Filtering
This approach identifies users who have similar preferences (based on ratings or clicks) and recommends items that similar users liked.

### Implementation
We can try to implement this method using matrix factorization techniques like **Singular Value Decomposition (SVD)**, which reduces the dimensionality of the data matrix, capturing latent factors that explain user-item interactions.


In [18]:
df_recommendation = pd.DataFrame(
    [{"id": item["parent_asin"], "user": item["user_id"], "rating": item["rating"]} for item in data])


In [19]:
df_recommendation.head(5)

,id,user,rating
0,B07TDSJZMR,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,4.0
1,B08637FWWF,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,5.0
2,B07KJVGNN5,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,5.0
3,B092RP73CX,AEZGPLOYTSAPR3DHZKKXEFPAXUAA,4.0
4,B08KYJLF5T,AEQAYV7RXZEBXMQIQPL6KCT2CFWQ,1.0


In [20]:
print(f"The number of unique products is : {df_recommendation.id.nunique()}")
print(f"The number of unique users is : {df_recommendation.user.nunique()}")
print(f"The number of products rated by different users)


The number of unique products is : 60274
The number of unique users is : 461656
The number of products rated by different users is: 488188


In [24]:
ratings_per_product = df_recommendation.groupby('id')['user'].nunique()
print(f"The proportion of products rated by different users : \n")
print(ratings_per_product.describe())

## Sur l'echantillon selectionné on remarque que plus de 50% des produits ne sont évalué que par une deux personnes ... 


The proportion of products rated by different users : 

count    60274.000000
mean         8.099479
std         45.797194
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max       3147.000000
Name: user, dtype: float64


### b) Item-Based Collaborative Filtering
Instead of focusing on user similarity, this method finds items that are similar based on user ratings or interactions.

# II- Content based filtering 

# III - Hybrid Recommender Systems 